In [75]:
import win32com.client
import pandas as pd
from datetime import datetime
from datetime import timedelta
instCpCybos = win32com.client.Dispatch('CpUtil.CpCybos')
instCpStockCode = win32com.client.Dispatch('CpUtil.CpStockCode')
instCpCodeMgr = win32com.client.Dispatch('CpUtil.CpCodeMgr')
instStockChart = win32com.client.Dispatch('CpSysDib.StockChart')


In [76]:
# 크레온 API 연결 확인
# <리턴값 종류 확인 필요>
def check_connection():
    if instCpCybos.IsConnect == 1:
        print('정상적으로 연결')
    else:
        print('연결 실패')
        
check_connection()

정상적으로 연결


In [77]:
# 종목 이름으로 찾기
def search_by_name(stock_name):
    count = instCpStockCode.GetCount()
    for i in range(count):
        if instCpStockCode.GetData(1,i) == stock_name:
            # GetData(x,y)
            # x : 0 => 종목 코드
            # x : 1 => 종목명
            # x : 2 => ISIN 코드(ISIN : 국제 증권 식별 번호)
            return [instCpStockCode.GetData(0,i), instCpStockCode.GetData(1,i)]

search_by_name('삼성전자')

['A005930', '삼성전자']

In [78]:
# 특정 코드의 종목 찾기
def search_by_code(stock_code):
    count = instCpStockCode.GetCount()
    for i in range(count):
        if instCpStockCode.GetData(0,i) == stock_code:
            # GetData(x,y)
            # x : 0 => 종목 코드
            # x : 1 => 종목명
            # x : 2 => ISIN 코드(ISIN : 국제 증권 식별 번호)
            return [instCpStockCode.GetData(0,i), instCpStockCode.GetData(1,i)]

search_by_code('A005930')

['A005930', '삼성전자']

In [79]:
# 종목명 전체 리스트 가져오기
# <선물이랑 ETF 쪽은 구분 필요>
def get_stock_list():
    count = instCpStockCode.GetCount()
    stock_list = []
    
    for i in range(count):
        stock_list.append(instCpStockCode.GetData(1,i))
    
    return stock_list

# get_stock_list()

In [80]:
# print(instCpCodeMgr.GetStockListByMarket(1))

In [81]:
# 전체 종목 가져오기
codelist = instCpCodeMgr.GetStockListByMarket(1)
kospi = {}
for code in codelist:
    name = instCpCodeMgr.CodeToName(code)
    kospi[code] = name
    
# kospi

In [82]:
# 
for i, code in enumerate(codelist):
    secondCode = instCpCodeMgr.GetStockSectionKind(code)
    name = instCpCodeMgr.CodeToName(code)
    # print(i, code, secondCode, name)

In [83]:
# 
code_list = []
for i, code in enumerate(codelist):
    secondCode = instCpCodeMgr.GetStockSectionKind(code)
    name = instCpCodeMgr.CodeToName(code)
    code_list.append(code)
# print(code_list)


In [84]:
import time
# 특정 범위 일자 종목 데이터 가져오기
def get_stock_info(stock_code, start_day, end_day, type):
    
    instStockChart.SetInputValue(0, stock_code) # 종목명
    instStockChart.SetInputValue(1, ord('1')) # 1 : 기간으로 요청, 2: 개수로 요청
    instStockChart.SetInputValue(3, start_day) # 요청 시작일
    instStockChart.SetInputValue(2, end_day) # 요청 종료일
    # instStockChart.SetInputValue(4, request_num) # 요청 개수
    '''
    # 요청할 데이터 종류(리스트 형태로 요청 가능)
    0 : 날짜, 1 : 시간 - hhmm, 2 : 시가, 3 : 고가, 4 : 저가, 5 : 종가
    6 : 전일대비, 8 : 거래량, 9 : 거래대금, 13 : 시가총액
    '''
    instStockChart.SetInputValue(5, [0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 12, 13, 14, 15, 16, 17,18, 19,20,21,22,23,24,25,26])
    instStockChart.SetInputValue(6, ord(type)) # 'D' : 일봉, 'm' : 분봉
    instStockChart.SetInputValue(9, ord('1'))

    instStockChart.BlockRequest() # 위 정보로 요청

    numrow, numcolumn = instStockChart.GetHeaderValue(3), instStockChart.GetHeaderValue(2)

    index = []
    for i in range(numrow):
        index_ = str(instStockChart.GetDataValue(0,i))
        index.append(index_)

    stock_info = pd.DataFrame(columns=numcolumn[1:], index=index)

    for num in range(numrow):
        for col in range(len(numcolumn)):
            # 1,2,3,4,5,6,7,8,9, 10
            stock_info.iloc[num, col-1] = str(instStockChart.GetDataValue(col,num))
    
    return stock_info

donghwa = get_stock_info('A005930', '20221029', '20221031', 'm')

In [85]:
import time
# 전체 일자 분봉 종목 데이터 가져오기
def get_stock_info_auto(stock_code, end_day, type):
    
    day_format = '%Y%m%d'
    minus_day = timedelta(days=5)
    
    transfer_end_day = datetime.strptime(end_day, day_format) # end_day 변환
    pre_day = datetime.strftime(transfer_end_day - minus_day, day_format) # pre_day 계산
    
    stock_df = get_stock_info(stock_code, pre_day, end_day, type) # 첫번째 DF 생성

    for day in range(150): # range 범위 수정하지 말 것
        time.sleep(0.3)
        dayily_df = pd.DataFrame()
        transfer_pre_day = datetime.strptime(pre_day, day_format)
        next_pre_day = datetime.strftime(transfer_pre_day - minus_day, day_format)

        info_second = get_stock_info(stock_code, next_pre_day, pre_day, type)
        
        pre_day = next_pre_day
        
        stock_df = pd.concat([stock_df, info_second], axis=0)
    
    stock_name = search_by_code(stock_code)
    print(stock_name, '남은 요청 횟수 :{0}'.format(instCpCybos.GetLimitRemainCount(0)))
    stock_df.to_csv('../data/{0}_{1}.csv'.format(stock_name[0][1:],stock_name[1]), encoding='utf-8-sig')

    return stock_df

# samsung = get_stock_info_auto('A005930', '20221101', 'm')
# samsung

In [86]:
import time
for code in code_list[60:]:
    get_stock_info_auto(code, '20221101', 'm')


['A000020', '동화약품'] 남은 요청 횟수 :20
['A000040', 'KR모터스'] 남은 요청 횟수 :20
['A000050', '경방'] 남은 요청 횟수 :20
['A000060', '메리츠화재'] 남은 요청 횟수 :20
['A000070', '삼양홀딩스'] 남은 요청 횟수 :20
['A000075', '삼양홀딩스우'] 남은 요청 횟수 :20
['A000080', '하이트진로'] 남은 요청 횟수 :20
['A000087', '하이트진로2우B'] 남은 요청 횟수 :20
['A000100', '유한양행'] 남은 요청 횟수 :20
['A000105', '유한양행우'] 남은 요청 횟수 :20
['A000120', 'CJ대한통운'] 남은 요청 횟수 :20
['A000140', '하이트진로홀딩스'] 남은 요청 횟수 :20
['A000145', '하이트진로홀딩스우'] 남은 요청 횟수 :20
['A000150', '두산'] 남은 요청 횟수 :20
['A000155', '두산우'] 남은 요청 횟수 :20
['A000157', '두산2우B'] 남은 요청 횟수 :20
['A000180', '성창기업지주'] 남은 요청 횟수 :20
['A000210', 'DL'] 남은 요청 횟수 :20
['A000215', 'DL우'] 남은 요청 횟수 :20
['A000220', '유유제약'] 남은 요청 횟수 :20
['A000225', '유유제약1우'] 남은 요청 횟수 :20
['A000227', '유유제약2우B'] 남은 요청 횟수 :20
['A000230', '일동홀딩스'] 남은 요청 횟수 :20
['A000240', '한국앤컴퍼니'] 남은 요청 횟수 :20
['A000270', '기아'] 남은 요청 횟수 :20
['A000300', '대유플러스'] 남은 요청 횟수 :20
['A000320', '노루홀딩스'] 남은 요청 횟수 :20
['A000325', '노루홀딩스우'] 남은 요청 횟수 :20
['A000370', '한화손해보험'] 남은 요청 횟수 :20
['A000390', '삼

KeyboardInterrupt: 

In [ ]:
instCpCybos.GetLimitRemainCount(0)

20